In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("/kaggle/input/mlpr-merged-data/storm_and_outage_merged.csv")

In [2]:
prism = pd.read_csv("/kaggle/input/prism-us-dataset-2014-23/prism_us_data.csv")

In [3]:
prism

,st_abb,st_code,date,stability,tmin,tmax,tavg,ppt
0,AL,1,20140101,stable,1.120,7.147,4.133,0.613
1,AL,1,20140102,stable,1.991,10.963,6.477,5.808
2,AL,1,20140103,stable,-5.715,11.507,2.896,3.426
3,AL,1,20140104,stable,-6.232,2.895,-1.669,0.005
4,AL,1,20140105,stable,-3.563,9.385,2.911,0.001
...,...,...,...,...,...,...,...,...
175291,WY,56,20231227,stable,-10.787,-0.377,-5.582,0.266
175292,WY,56,20231228,stable,-10.978,1.837,-4.570,0.002
175293,WY,56,20231229,stable,-10.555,3.532,-3.511,0.001
175294,WY,56,20231230,stable,-8.899,5.913,-1.493,0.005


In [4]:
df

,Unnamed: 0,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,...,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,EPISODE_NARRATIVE,EVENT_NARRATIVE,MAGNITUDE_IMPUTED,MAGNITUDE_TYPE_IMPUTED
0,0,4.0,2014-11-23 12:15:00,2014-11-23 12:05:00,2014-11-23 12:25:00,0.333333,ALABAMA,1.0,Thunderstorm Wind,C,...,0.0,0.0,0.0,3000.0,0.0,Emergency Manager,A strong upper level trough and surface cold f...,Numerous trees were blown down across the county.,55.00000,EG
1,1,65.0,2014-12-23 15:45:00,2014-12-23 15:44:00,2014-12-23 15:45:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,...,0.0,0.0,0.0,0.0,0.0,Amateur Radio,A deep upper level trough over the Plains Stat...,Several large tree limbs broken.,50.00000,EG
2,2,53.0,2014-12-23 17:00:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,...,0.0,0.0,0.0,0.0,0.0,Emergency Manager,A deep upper level trough over the Plains Stat...,Ten to fifteen roads in the town of Vernon wer...,0.92894,MS
3,3,47.0,2014-12-23 17:15:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,...,0.0,0.0,0.0,0.0,0.0,Emergency Manager,A deep upper level trough over the Plains Stat...,Ten to fifteen roads in the town of Vernon wer...,0.92894,MS
4,4,33.0,2014-12-23 19:30:00,2014-12-23 19:30:00,2014-12-23 19:31:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,...,0.0,0.0,0.0,1000.0,0.0,Public,Strong to isolated severe thunderstorms develo...,Strong wind gusts near 50 mph uprooted a magno...,45.00000,EG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407006,407006,5807.0,2023-12-22 23:30:00,2023-12-22 23:20:00,2023-12-22 23:45:00,0.416667,PUERTO RICO,99.0,Flash Flood,C,...,0.0,0.0,0.0,1100.0,0.0,Emergency Manager,"A wet pattern peaked Friday, December 22nd, an...",Emergency Management reported 4 people trapped...,0.32800,EG
407007,407007,2707.0,2023-12-22 23:45:00,2023-12-22 23:20:00,2023-12-22 23:45:00,0.416667,PUERTO RICO,99.0,Flash Flood,C,...,0.0,0.0,0.0,1100.0,0.0,Emergency Manager,"A wet pattern peaked Friday, December 22nd, an...",Emergency Management reported 4 people trapped...,0.32800,EG
407008,407008,706.0,2023-12-23 01:30:00,2023-12-23 01:30:00,2023-12-23 02:00:00,0.500000,PUERTO RICO,99.0,Flood,C,...,0.0,0.0,0.0,400.0,0.0,911 Call Center,"A wet pattern peaked Friday, December 22nd, an...","A 911 employee reported a flooded road, parti...",0.25750,EG
407009,407009,757.0,2023-12-23 01:45:00,2023-12-23 01:30:00,2023-12-23 02:00:00,0.500000,PUERTO RICO,99.0,Flood,C,...,0.0,0.0,0.0,400.0,0.0,911 Call Center,"A wet pattern peaked Friday, December 22nd, an...","A 911 employee reported a flooded road, parti...",0.25750,EG


In [5]:
df['date'] = pd.to_datetime(df['power_outage_datetime']).dt.date

# Convert the 'date' column in prism to a date format (ignoring time)
prism['date'] = pd.to_datetime(prism['date'].astype(str), format='%Y%m%d').dt.date

# Merge df and prism on the state codes and date
# 'STATE_FIPS' from df matches with 'st_code' from prism
prism_noaa = pd.merge(df, prism, left_on=['STATE_FIPS', 'date'], right_on=['st_code', 'date'], how='left')

# Verify the result by displaying the first few rows
prism_noaa

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,...,MAGNITUDE_IMPUTED,MAGNITUDE_TYPE_IMPUTED,date,st_abb,st_code,stability,tmin,tmax,tavg,ppt
0,0,4.0,2014-11-23 12:15:00,2014-11-23 12:05:00,2014-11-23 12:25:00,0.333333,ALABAMA,1.0,Thunderstorm Wind,C,...,55.00000,EG,2014-11-23,AL,1.0,stable,6.155,19.191,12.673,3.333
1,1,65.0,2014-12-23 15:45:00,2014-12-23 15:44:00,2014-12-23 15:45:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,...,50.00000,EG,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036
2,2,53.0,2014-12-23 17:00:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,...,0.92894,MS,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036
3,3,47.0,2014-12-23 17:15:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,...,0.92894,MS,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036
4,4,33.0,2014-12-23 19:30:00,2014-12-23 19:30:00,2014-12-23 19:31:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,...,45.00000,EG,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407006,407006,5807.0,2023-12-22 23:30:00,2023-12-22 23:20:00,2023-12-22 23:45:00,0.416667,PUERTO RICO,99.0,Flash Flood,C,...,0.32800,EG,2023-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407007,407007,2707.0,2023-12-22 23:45:00,2023-12-22 23:20:00,2023-12-22 23:45:00,0.416667,PUERTO RICO,99.0,Flash Flood,C,...,0.32800,EG,2023-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407008,407008,706.0,2023-12-23 01:30:00,2023-12-23 01:30:00,2023-12-23 02:00:00,0.500000,PUERTO RICO,99.0,Flood,C,...,0.25750,EG,2023-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407009,407009,757.0,2023-12-23 01:45:00,2023-12-23 01:30:00,2023-12-23 02:00:00,0.500000,PUERTO RICO,99.0,Flood,C,...,0.25750,EG,2023-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
prism_noaa = prism_noaa.dropna(subset=['ppt'])

In [7]:
df = prism_noaa

In [ ]:
# df = df[df['EVENT_NARRATIVE'].notnull()]
df['description'] = df['EVENT_NARRATIVE'].fillna('') + ' ' + df['EPISODE_NARRATIVE'].fillna('')   

/tmp/ipykernel_31/2737538787.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['EVENT_NARRATIVE'].fillna('') + ' ' + df['EPISODE_NARRATIVE'].fillna('')


In [9]:
# df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['Unnamed: 0','EVENT_NARRATIVE','EPISODE_NARRATIVE'])

In [10]:
df.isnull().sum()

customers_out               0
power_outage_datetime       0
storm_start_datetime_est    0
storm_end_datetime_est      0
duration_hours              0
STATE                       0
STATE_FIPS                  0
EVENT_TYPE                  0
CZ_TYPE                     0
CZ_FIPS                     0
CZ_NAME                     0
WFO                         0
INJURIES_DIRECT             0
INJURIES_INDIRECT           0
DEATHS_DIRECT               0
DEATHS_INDIRECT             0
DAMAGE_PROPERTY             0
DAMAGE_CROPS                0
SOURCE                      0
MAGNITUDE_IMPUTED           0
MAGNITUDE_TYPE_IMPUTED      0
date                        0
st_abb                      0
st_code                     0
stability                   0
tmin                        0
tmax                        0
tavg                        0
ppt                         0
description                 0
dtype: int64

In [11]:
df

,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,CZ_FIPS,...,MAGNITUDE_TYPE_IMPUTED,date,st_abb,st_code,stability,tmin,tmax,tavg,ppt,description
0,4.0,2014-11-23 12:15:00,2014-11-23 12:05:00,2014-11-23 12:25:00,0.333333,ALABAMA,1.0,Thunderstorm Wind,C,45.0,...,EG,2014-11-23,AL,1.0,stable,6.155,19.191,12.673,3.333,Numerous trees were blown down across the coun...
1,65.0,2014-12-23 15:45:00,2014-12-23 15:44:00,2014-12-23 15:45:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,119.0,...,EG,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036,Several large tree limbs broken. A deep upper ...
2,53.0,2014-12-23 17:00:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,75.0,...,MS,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036,Ten to fifteen roads in the town of Vernon wer...
3,47.0,2014-12-23 17:15:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,75.0,...,MS,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036,Ten to fifteen roads in the town of Vernon wer...
4,33.0,2014-12-23 19:30:00,2014-12-23 19:30:00,2014-12-23 19:31:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,3.0,...,EG,2014-12-23,AL,1.0,stable,9.554,13.836,11.695,7.036,Strong wind gusts near 50 mph uprooted a magno...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405923,3.0,2023-12-26 01:15:00,2023-12-26 00:26:00,2023-12-26 11:30:00,11.066667,WYOMING,56.0,Blizzard,Z,116.0,...,MS,2023-12-26,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet stations Summit East (WY28) loca...
405924,3498.0,2023-12-26 03:45:00,2023-12-26 03:44:00,2023-12-26 08:26:00,4.700000,WYOMING,56.0,High Wind,Z,107.0,...,MG,2023-12-26,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet station Coleman (WY1) along Inte...
405925,3498.0,2023-12-26 04:00:00,2023-12-26 03:44:00,2023-12-26 08:26:00,4.700000,WYOMING,56.0,High Wind,Z,107.0,...,MG,2023-12-26,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet station Coleman (WY1) along Inte...
405926,1513.0,2023-12-26 04:15:00,2023-12-26 03:44:00,2023-12-26 08:26:00,4.700000,WYOMING,56.0,High Wind,Z,107.0,...,MG,2023-12-26,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet station Coleman (WY1) along Inte...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395267 entries, 0 to 405927
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   customers_out             395267 non-null  float64
 1   power_outage_datetime     395267 non-null  object 
 2   storm_start_datetime_est  395267 non-null  object 
 3   storm_end_datetime_est    395267 non-null  object 
 4   duration_hours            395267 non-null  float64
 5   STATE                     395267 non-null  object 
 6   STATE_FIPS                395267 non-null  float64
 7   EVENT_TYPE                395267 non-null  object 
 8   CZ_TYPE                   395267 non-null  object 
 9   CZ_FIPS                   395267 non-null  float64
 10  CZ_NAME                   395267 non-null  object 
 11  WFO                       395267 non-null  object 
 12  INJURIES_DIRECT           395267 non-null  float64
 13  INJURIES_INDIRECT         395267 non-null  float6

## Storm Prediction approach
After Imputing the Magnitude values, we can now create a new column 'is_storm' to classify storms based on the magnitude threshold, which was found to be above 25 through the distribution of magnitude values. Using this threshold, we can create a new 'is_storm' column in the DataFrame and mark all of them as 1.


In [13]:
# Create a new column 'is_storm' based on MAGNITUDE_IMPUTED values
df['is_storm'] = 0
df.loc[df['MAGNITUDE_IMPUTED'] >= 25, 'is_storm'] = 1

# Display counts and percentages of each category
storm_counts = df['is_storm'].value_counts()
storm_percentages = df['is_storm'].value_counts(normalize=True) * 100

# Create a summary dataframe
summary_df = pd.DataFrame({
    'Count': storm_counts,
    'Percentage': storm_percentages.round(2)
})

print("Storm classification summary:")
print(summary_df)

Storm classification summary:
           Count  Percentage
is_storm                    
0         205239       51.92
1         190028       48.08


## Handling No Storms

We were not taking no storms into account in the initial classification models that we implemented which categorized between the type of storm (low, medium, High). To handle no storms, we can create a new category 'No Storm' in the 'is_storm' column and assign it to all Records where the Magnitude is below 25

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
"""
    Task:
    Classify storm severity based on imputed magnitude values and prepare data for machine learning.

    Approach:
    1. Filter magnitude values above 25 and calculate 33rd and 66th percentiles.
    2. Define a classification function to assign severity classes (No Storm, Low, Moderate, High) based on magnitude thresholds.
    3. Apply the classification to create a new severity class column in the DataFrame.
    """
# Make a copy of the DataFrame (assuming df is already defined)
storm_df = df.copy()

filtered_scores = storm_df[storm_df['MAGNITUDE_IMPUTED'] > 25]['MAGNITUDE_IMPUTED']

# Calculate percentiles
p33 = filtered_scores.quantile(0.33)
p66 = filtered_scores.quantile(0.66)

# Define severity classification based on percentiles
def classify(score):
    """
    Task:
    Classify storm severity based on a given magnitude score using predefined thresholds.

    Parameters:
    score : float or int
        The imputed magnitude score to classify.

    Returns:
    int
        Severity class (10 for No Storm, 0 for Low, 1 for Moderate, 2 for High).
    """
    if score <= 25:
        return 10 # No Storm
    elif score <= p33: # 49
        return 0  # Low
    elif score <= p66: # 54
        return 1  # Moderate
    else:
        return 2  # High

# Create severity_class column
storm_df['severity_class'] = storm_df['MAGNITUDE_IMPUTED'].apply(classify)

# label_encoder = LabelEncoder()
# storm_df['severity_encoded'] = label_encoder.fit_transform(storm_df['severity_class'])

In [15]:
storm_df

,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,CZ_FIPS,...,st_abb,st_code,stability,tmin,tmax,tavg,ppt,description,is_storm,severity_class
0,4.0,2014-11-23 12:15:00,2014-11-23 12:05:00,2014-11-23 12:25:00,0.333333,ALABAMA,1.0,Thunderstorm Wind,C,45.0,...,AL,1.0,stable,6.155,19.191,12.673,3.333,Numerous trees were blown down across the coun...,1,2
1,65.0,2014-12-23 15:45:00,2014-12-23 15:44:00,2014-12-23 15:45:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,119.0,...,AL,1.0,stable,9.554,13.836,11.695,7.036,Several large tree limbs broken. A deep upper ...,1,1
2,53.0,2014-12-23 17:00:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,75.0,...,AL,1.0,stable,9.554,13.836,11.695,7.036,Ten to fifteen roads in the town of Vernon wer...,0,10
3,47.0,2014-12-23 17:15:00,2014-12-23 17:00:00,2014-12-23 19:30:00,2.500000,ALABAMA,1.0,Flash Flood,C,75.0,...,AL,1.0,stable,9.554,13.836,11.695,7.036,Ten to fifteen roads in the town of Vernon wer...,0,10
4,33.0,2014-12-23 19:30:00,2014-12-23 19:30:00,2014-12-23 19:31:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,3.0,...,AL,1.0,stable,9.554,13.836,11.695,7.036,Strong wind gusts near 50 mph uprooted a magno...,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405923,3.0,2023-12-26 01:15:00,2023-12-26 00:26:00,2023-12-26 11:30:00,11.066667,WYOMING,56.0,Blizzard,Z,116.0,...,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet stations Summit East (WY28) loca...,0,10
405924,3498.0,2023-12-26 03:45:00,2023-12-26 03:44:00,2023-12-26 08:26:00,4.700000,WYOMING,56.0,High Wind,Z,107.0,...,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet station Coleman (WY1) along Inte...,1,2
405925,3498.0,2023-12-26 04:00:00,2023-12-26 03:44:00,2023-12-26 08:26:00,4.700000,WYOMING,56.0,High Wind,Z,107.0,...,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet station Coleman (WY1) along Inte...,1,2
405926,1513.0,2023-12-26 04:15:00,2023-12-26 03:44:00,2023-12-26 08:26:00,4.700000,WYOMING,56.0,High Wind,Z,107.0,...,WY,56.0,stable,-15.951,-4.296,-10.124,0.650,WYDOT mesonet station Coleman (WY1) along Inte...,1,2


In [16]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

storm_df['desc_clean'] = storm_df['description'].astype(str).apply(preprocess)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
storm_df['desc_word_count'] = storm_df['desc_clean'].apply(lambda x: len(x.split()))
storm_df['desc_char_count'] = storm_df['desc_clean'].apply(len)

In [ ]:
keywords = ['tornado', 'hail', 'flood', 'wind', 'tree', 'power', 'damage', 'outage', 'broken', 'blown']

for word in keywords:
    storm_df[f'has_{word}'] = storm_df['desc_clean'].apply(lambda x: 1 if word in x else 0)


# We are increasing the number of features by adding new columns based on the keywords like tornado, hail, flood, wind, tree, power, damage, outage, broken, and blown.
# These columns will have a value of 1 if the keyword is present in the description and 0 otherwise.

In [19]:
storm_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395267 entries, 0 to 405927
Data columns (total 45 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   customers_out             395267 non-null  float64
 1   power_outage_datetime     395267 non-null  object 
 2   storm_start_datetime_est  395267 non-null  object 
 3   storm_end_datetime_est    395267 non-null  object 
 4   duration_hours            395267 non-null  float64
 5   STATE                     395267 non-null  object 
 6   STATE_FIPS                395267 non-null  float64
 7   EVENT_TYPE                395267 non-null  object 
 8   CZ_TYPE                   395267 non-null  object 
 9   CZ_FIPS                   395267 non-null  float64
 10  CZ_NAME                   395267 non-null  object 
 11  WFO                       395267 non-null  object 
 12  INJURIES_DIRECT           395267 non-null  float64
 13  INJURIES_INDIRECT         395267 non-null  float6

In [20]:
# output_file = '/kaggle/working/noaa+powout+prism.csv'
# storm_df.to_csv(output_file)

# Dealing with lag (Predicting Storm 1 hour ahead)

The Datetime column is doesnt have equal intervals, so to handle this we create an algorithm where we find the closest Variable atleast 1 hour ahead of the current row. Though States are also grouped by to not alter records as 2 storms might be happening in different states at the same time.

In [ ]:
import pandas as pd

# Ensure power_outage_datetime is in datetime format
storm_df['power_outage_datetime'] = pd.to_datetime(storm_df['power_outage_datetime'])

# Sort the DataFrame by state and datetime to ensure temporal order
storm_df = storm_df.sort_values(by=['STATE_FIPS', 'power_outage_datetime'])

# Function to apply a 1-hour lag within each state group
def apply_1hour_lag(group):
    """
    Task:
    Create a lagged storm indicator by assigning the 'is_storm' value from the closest row at least 1 hour ahead within the same state group.

    Parameters:
    group : pandas.DataFrame
        A DataFrame group (e.g., grouped by 'STATE_FIPS') with 'power_outage_datetime' and 'is_storm' columns.

    Returns:
    pandas.DataFrame
        The group with a new 'is_storm_lagged' column containing the lagged 'is_storm' values.
    """
    # Initialize the lagged column
    group['is_storm_lagged'] = None
    
    # Iterate through the group to find the appropriate row for 1-hour lag
    for i in range(len(group)):
        current_time = group.iloc[i]['power_outage_datetime']
        future_time = current_time + pd.Timedelta(hours=1)
        
        # Find the closest row within the same state that is at least 1 hour ahead
        future_rows = group[(group['power_outage_datetime'] >= future_time) & 
                           (group['power_outage_datetime'] <= future_time + pd.Timedelta(hours=1))]
        
        if not future_rows.empty:
            # Take the first row (closest to 1 hour)
            group.iloc[i, group.columns.get_loc('is_storm_lagged')] = future_rows.iloc[0]['is_storm']
    
    return group

# Apply the lagging function to each state group
storm_df = storm_df.groupby('STATE_FIPS').apply(apply_1hour_lag).reset_index(drop=True)

# Drop rows where is_storm_lagged is null
storm_df = storm_df.dropna(subset=['is_storm_lagged'])

# Display the result
print(storm_df[['power_outage_datetime', 'STATE_FIPS', 'is_storm', 'is_storm_lagged']].head(10))

/tmp/ipykernel_31/2772570984.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  storm_df = storm_df.groupby('STATE_FIPS').apply(apply_1hour_lag).reset_index(drop=True)


   power_outage_datetime  STATE_FIPS  is_storm is_storm_lagged
1    2014-12-23 15:45:00         1.0         1               0
9    2014-12-27 21:00:00         1.0         0               0
14   2015-01-03 15:30:00         1.0         1               0
15   2015-01-03 17:30:00         1.0         0               0
25   2015-01-15 05:00:00         1.0         1               1
26   2015-01-15 05:15:00         1.0         1               1
27   2015-01-15 05:30:00         1.0         1               1
28   2015-01-15 05:45:00         1.0         1               1
29   2015-01-15 06:00:00         1.0         1               1
30   2015-01-15 06:15:00         1.0         1               1


In [24]:
storm_df

,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,CZ_FIPS,...,has_hail,has_flood,has_wind,has_tree,has_power,has_damage,has_outage,has_broken,has_blown,is_storm_lagged
1,65.0,2014-12-23 15:45:00,2014-12-23 15:44:00,2014-12-23 15:45:00,0.016667,ALABAMA,1.0,Thunderstorm Wind,C,119.0,...,0,0,0,1,0,0,0,1,0,0
9,70.0,2014-12-27 21:00:00,2014-12-27 21:00:00,2014-12-28 00:00:00,3.000000,ALABAMA,1.0,Heavy Rain,C,73.0,...,0,0,0,0,0,0,0,0,0,0
14,3.0,2015-01-03 15:30:00,2015-01-03 15:30:00,2015-01-03 15:30:00,0.000000,ALABAMA,1.0,Thunderstorm Wind,C,99.0,...,0,0,1,0,1,0,0,0,0,0
15,3.0,2015-01-03 17:30:00,2015-01-03 17:25:00,2015-01-03 18:25:00,1.000000,ALABAMA,1.0,Flash Flood,C,107.0,...,0,1,0,0,0,0,0,0,0,0
25,1.0,2015-01-15 05:00:00,2015-01-15 05:00:00,2015-01-15 10:00:00,5.000000,ALABAMA,1.0,Winter Weather,Z,4.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395253,3.0,2023-12-25 21:45:00,2023-12-25 21:36:00,2023-12-26 02:02:00,4.433333,WYOMING,56.0,Blizzard,Z,119.0,...,0,0,1,0,0,0,0,0,0,1
395254,3.0,2023-12-25 22:00:00,2023-12-25 21:36:00,2023-12-26 02:02:00,4.433333,WYOMING,56.0,Blizzard,Z,119.0,...,0,0,1,0,0,0,0,0,0,1
395256,2.0,2023-12-25 22:30:00,2023-12-25 22:10:00,2023-12-26 17:53:00,19.716667,WYOMING,56.0,High Wind,Z,118.0,...,0,0,1,0,0,0,0,0,0,0
395257,2.0,2023-12-25 22:45:00,2023-12-25 22:10:00,2023-12-26 17:53:00,19.716667,WYOMING,56.0,High Wind,Z,118.0,...,0,0,1,0,0,0,0,0,0,0


In [23]:
output_file = '/kaggle/working/noaa+powout+prism_0+1_Is_Storm_Lag.csv'
storm_df.to_csv(output_file)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
"""
    Task:
    Prepare and split data for training a model to predict lagged storm occurrence.

    """
# Define features (same as in your notebook)
features = [
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'customers_out', 'duration_hours',
    'desc_word_count', 'desc_char_count',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'has_power', 'has_damage', 'has_outage', 'has_broken', 'has_blown',
    'tmin', 'tmax', 'tavg', 'ppt'
]

# Prepare X and y
X = storm_df[features]
y = storm_df['is_storm_lagged'].astype(int)  # Ensure y is integer

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Train logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
"""
    Task:
    Train a Random Forest Classifier and evaluate its performance for storm prediction.

    Approach:
    1. Initialize and train a Random Forest Classifier with specified parameters.
    
    """
# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
import xgboost as xgb

# Calculate scale_pos_weight for imbalanced data
# scale_pos_weight = (number of negative samples) / (number of positive samples)
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1]) if len(y_train[y_train == 1]) > 0 else 1

# Train XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# from sklearn.svm import SVC

# # Train SVM model
# svm_model = SVC(kernel='rbf', class_weight='balanced', random_state=42)
# svm_model.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = svm_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)
# print("\nClassification Report:\n", classification_report(y_test, y_pred))
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))